<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/timaslj/Lab-3/timaslj_lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Timas Ljungdahl**

# **Abstract**

In this report, 4 different algorithms were implemented and tested. The algorithms were sparse matrix-vector multiplication for matrices of CRS format, modified Gram-Schmidt iteration for QR-factorization, backwards substitution for solving $Rx = b$ and finally QR eigenvalue algorithm for finding the eigenvalues and eigenvectors for a matrix $A$. All algorithms were implemented and tested with random data and generally generated results with around 10 decimal precision. The results are probably highly affected by floating point error that occur when continuously adding and subtracting numbers of different magnitude. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import unittest
import random
import math

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report, systems of linear equations are investigated of the form $Ax = b$
where we want to solve for $x$ given a matrix $A$ and vector $b$. A system of linear equations has an exact solution if A is nonsingular, meaning that A has an inverse, since then $x = A^{-1}b$. In this report we assume that A is a square, nonsingular matrix. 

The direct solution methods implemented in this report are based on factorization of A into easily invertable matrices - diagonal, orthonormal and triangular matrices. The problem of solving for $x$ is summerized below.

$Ax = b$, solve for $x$
1. Factorize $A$ into $QR$, where $Q$ is an orthonormal matrix and $R$ is an upper triangular matrix. This gives us $QRx = b$.
2. Since $Q$ is orthonormal $Q^{-1} = Q^T$. Orthonormal, means that each column vector in the matrix is normalized and orthogonal to each other. Multiplying $Q^{-1}$ on the left on each side, we get $Q^{-1}QRx = Q^{-1}b => Rx = Q^Tb$.
3. We can now easily solve $Rx = Q^Tb$ 

Apart from solving systems of equations, sparse matrix-vector multiplication was implemented for sparse matrices of compressed row storage(CRS)format. Instead of storing all values in the matrix, only the nonzero values are stored in an array ***v***, along with the two index arrays ***col_idx*** and ***row_ptr***. There is an entry in the ***col_idx*** array for each value to indicate the column of the value in the original matrix. The ***row_ptr*** array consists of the index in ***v*** where each row starts. 

The QR eigenvalue algorithm was also implementet which for a real symmetric matrix $A$, returns a unitary matrix $U$ with the eigenvectors of $A$ as column vectors and a upper triangular matrix $T$ with eigenvalues of $A$ in the diagonal. It finds the a Schur factorization such that $A = UTU^{*}$.

# **Methods**

To achieve the goal of solving the system of equations, the first step is to implement an effecient factorization method. In this report, a modified version of Gram-Schmidt iteration is implemented. This method recursively computes an orthonormal matrix $Q$ from $A$ by taking each column vector in $A$ and subtracting its projection onto the already computed orthonormal space in order to compute a new perpendicular vector which is then added to the set of orthonormal vectors. $R$ is then computed as $R = Q^TA$.  

When $Q$ and $R$ have been computed, $b$ is multiplied by $Q^T$ on the left to form a new vector $b_{prim}$ so that $Rx = b_{prim}$. This new system of equations can now be solved with backwards subsitution as $R$ is upper triangular. This is possible since $x_n = b_n/a_{nn}$ which can then be substituted in order to solve for $x_{n-1} = (b_{n-1} - a_{(n-1)(n)}x_n)/a_{(n-1)(n-1)}$. This can be written as:

$x_i = (b_i - \sum_{j=i+1}^{n}a_{ij}x_j)/a_{ii} $

In order to implement these algorithms, pseudocode from the lecture notes provided in class were followed. 

To test the QR factorization method,random $A \epsilon \mathbb{R}^{n \times n}$ were generated with imported numpy methods. The output $Q$ and $R$ of the algorithm were then multiplied together again and asserted to be equal to $A$.

To test the direct solver method, random $A \epsilon \mathbb{R}^{n \times n}$ and $b \epsilon \mathbb{R}^{n}$ were generated and the residuals $|| Ax-b ||$ and $|| x-y ||$ were asserted to be equal to zero. 

To test the QR eigenvalue algorithm, a random real symmetric matrix A was generated and $det(A - \lambda_iI)$ was asserted to be equal to 0. $||Av_i - \lambda_iv_i||$ was also asserted to be equal to 0. The tests follow the definition of eigenvectors and eigenvalues: $Av = \lambda v$ where $v$ is an eigenvector and $\lambda$ is an eigenvalue. 

# **Results**

In [0]:
##can be done without saving R,D and D_inverse -> much more space efficient
def jacobi_iteration(A,b, TOL = 1e-8):
  n = A.shape[0]  
  D_inverse = np.zeros((n,n))
  for i in range(n):
    D_inverse[i,i] = 1/A[i,i]

  M = np.identity(n) - np.matmul(D_inverse, A)

  c = np.matmul(D_inverse,b)

  x = np.random.rand(n)

  while np.linalg.norm(np.matmul(A,x)-b) > TOL:
    x = np.matmul(M, x) + c

  return x

def gauss_seidel_iteration(A, b, TOL = 1e-8):
  n = A.shape[0]  
  L_inverse = A.copy()
  for i in range(n):
    for j in range(n):
      if (j > i):
        L_inverse[i,j] = 0

  L_inverse = np.linalg.inv(L_inverse)

  M = np.identity(n) - np.matmul(L_inverse, A)

  c = np.matmul(L_inverse, b)

  x = np.random.rand(n)

  while np.linalg.norm(np.matmul(A,x)-b) > TOL:
    x = np.matmul(M, x) + c

  return x
    
class Test(unittest.TestCase):

  #left preconditioning so system converges
  def test_random_sparse_matrix(self):
    size = random.randint(2, 50) 
    A = np.random.rand(size, size)
    alpha = 1.1
    B = np.linalg.inv(A)*alpha #approx inverse of A so ||I-alpha*B*A|| < 1
    C = np.matmul(A,B)
    x = np.random.rand(size)
    b = np.matmul(C,x)

    x_approx = gauss_seidel_iteration(C, b)
    print(x_approx)
    print(x)

    #print(x)
    #x_prim = jacobi_iteration(A,b)
    
    
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

[0.59375668 0.41458276 0.7088139  0.03278796 0.43836699 0.12414629
 0.23843034 0.97403904 0.8738789  0.61685779 0.44859183 0.64692526
 0.04297032 0.32615792 0.55055631 0.98184408 0.44004754 0.82152972
 0.99683997 0.13093163 0.78075986 0.98283926 0.84871536 0.47386535
 0.53184105 0.80384882 0.76745939 0.56019984 0.07798569 0.40709451
 0.57322915 0.27859381 0.04679653 0.92919737 0.91002934 0.20163357]
[0.59375668 0.41458276 0.7088139  0.03278796 0.43836699 0.12414629
 0.23843034 0.97403904 0.8738789  0.61685779 0.44859183 0.64692526
 0.04297032 0.32615792 0.55055631 0.98184408 0.44004754 0.82152972
 0.99683997 0.13093163 0.78075986 0.98283926 0.84871536 0.47386535
 0.53184105 0.80384882 0.76745939 0.56019984 0.07798569 0.40709451
 0.57322915 0.27859381 0.04679653 0.92919737 0.91002934 0.20163357]



----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


In [969]:
def get_random_func(n):
  exponents = []
  #func_string = ''
  scalar = random.randint(0,1000)
  for i in range(n):
    exp = random.randint(1,5)
    exponents.append(exp)
    #func_string += 'x' + str(i) + '^' + str(exp) + ' + ' 
  #print(func_string + str(scalar))

  def random_func(x):
    sum = 0
    for i in range(n):
      sum += x[i]**exponents[i]
    return sum + scalar
  return random_func

def scalar_jacobian(f, x, dx=1e-8):
  fx = f(x)
  J = np.zeros(x.shape[0])
  dxi = x.copy()

  for i in range(len(x)):
    dxi[i] = dxi[i] + dx
    J[i] = abs(f(dxi)-fx)/dx 
    dxi[i] = x[i]
  
  return J

def scalar_newtons_method(f, x0, TOL = 1e-8):
  x = x0

  while abs(f(x)) > TOL:
    df = jacobian(f, x)
    for i in range(x.shape[0]): 
      if not(math.isclose(df[i],0)): #divide by zero
        x[i] -= f(x)/df[i]
      else:
        return None 
      
  return x

class Test(unittest.TestCase):

  #left preconditioning so system converges
  def test_random_functions(self):
    for n in range(1000):
      size = random.randint(2,10)
      f = get_random_func(size)
      x0 = np.zeros(size, dtype='float64')

      root_approx = scalar_newtons_method(f,x0)

      if root_approx is None:
        continue

      f_root = f(root_approx)
    
      if f_root == f_root: #not NaN
        self.assertAlmostEqual(f_root, 0, 5)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)





.
----------------------------------------------------------------------
Ran 1 test in 0.101s

OK


In [1080]:
def vector_jacobian(f, x, dx=1e-6):
  fx = f(x)
  number_of_variables = fx[1]
  J = np.zeros((fx[0].shape[0], number_of_variables), dtype='float64')
  dxi = x.copy()

  for i in range(x.shape[0]):
    dxi[i] = dxi[i] + dx
    fdxi = f(dxi)
    J[:,i] = (fdxi[0]-fx[0])/dx 
    dxi[i] = x[i]
  
  return J

def vector_newtons_method(f, x0, TOL = 1e-8):
  x = x0

  fx = f(x)

  nums_of_its = 0 #ta bort

  while np.linalg.norm(fx[0]) > TOL:
    fx = f(x)
    df = vector_jacobian(f,x)
    dx = np.linalg.solve(df, -fx[0])
    x = x + dx
    nums_of_its += 1

  print("Nums of its: ", nums_of_its)
  return x

def function1(x):
  return (np.array([(x[0]**2)*x[1], 5*x[0]+math.sin(x[1])], dtype='float64'),2)

def function2(x):
  return (np.array([x[0]*math.cos(x[1]), x[0]*math.sin(x[1])], dtype='float64'),2)

def function3(x):
  return (np.array([5*x[1], 4*(x[0]**2)*2*math.sin(x[1]*x[2]), x[1]*x[2]], dtype='float64'),3)

def get_random_vector_func():
  number_of_vars = random.randint(2,10)
  exponents = []
  scalars = []
  for funcs in range(number_of_vars):
    func_string = ''
    scalars.append(random.randint(0,100))
    func_exponents = []
    for i in range(number_of_vars):
      func_exponents.append(random.randint(1,10))
      func_string += 'x' + str(i) + '^' + str(func_exponents[i]) + ' + ' 
    print(func_string + str(scalars[funcs]))

    exponents.append(func_exponents)
  
  def random_vector_func(x):
    fx = np.zeros(number_of_vars)

    for funcs in range(number_of_vars):
      sum = 0
      for i in range(number_of_vars):
        sum += x[i]**func_exponents[funcs][i]
      fx[funcs] = sum + scalar
  
  return (random_vector_func, number_of_vars)


#print(root_approx)
root_approx=vector_newtons_method(function1, np.array([50,100000], dtype='float64'))
print(root_approx)
print(function1(root_approx))
#fx = get_random_vector_func()
#x0 = np.zeros(fx[1], dtype='float64')
#root_approx = vector_newtons_method(fx, x0)
#print(function3(root_approx))




Nums of its:  31
[2.48871078e-07 1.00172823e+05]
(array([ 6.20438549e-09, -1.35111465e-12]), 2)


In [0]:


print(jacobian(np.array([[2,5],[8,-1]]), np.array([1,2])))

# **Discussion**

All algorithms were implemented and tested with random data several times. The precision of the algorithms differs but generally the output had a precision of around 10 decimals. The precision is probabaly affected by floating point errors that occur when adding and subtracting numbers of different magnitude. The modified Graham-Schmidt iteration, however, mitigates the absorption effect of floating point addition as the orthonormal set is generated recursively and does not rely on the summation of a large set of numbers.    